# Scrape various features from RYM

Use the below to:
- Sign in to RYM
- Collect album info / meta-data

Note that Rateyourmusic.com has strict protection against scraping. Setting time delays and logging-in is critical to avoid being blocked.

In [ ]:
from selenium import webdriver
import requests
from tqdm import tqdm_notebook as tqdm
import time
import pandas as pd
import random

# 1. Collect descriptors and album-level information

In [ ]:
def open_and_login(user, pword, sleep):

    #Start at login and take user through to homepage, logged in
    #Make sure to give real sleep times to help glitches
    #Make sure ChromeDriver is in correct filepath
    sign_on = 'https://rateyourmusic.com/account/login'
    driver = webdriver.Chrome('./chromedriver1')
    driver.get(sign_on)
    
    #input user
    username = driver.find_element_by_id("username")
    username.clear()
    username.send_keys(user)

    #input pword
    password = driver.find_element_by_name("password")
    password.clear()
    password.send_keys(pword)

    #login
    driver.find_element_by_id("login_submit").click()

    time.sleep(sleep)

    #close pop ups - may not always be necessary
    driver.find_element_by_class_name("as-oil-l-item").click()

    time.sleep(sleep)
    
    return driver


In [ ]:
def navigate_to_charts(pause, driver):

    driver.find_element_by_xpath('//*[@title="View top charts by year, location, genre and more!"]').click()

    time.sleep(pause)

In [ ]:
def grab_album_metadata(num_pages, driver):
    
    album_name = []
    artist_name = []
    descriptor = []
    album_id = []
    album_genre = []
    
    for page in tqdm(range(num_pages)):

        album_tags = driver.find_elements_by_tag_name('td')

        for album in album_tags:

        # album name
            try:
                album_name.append(album.find_element_by_class_name('chart_detail_line2').find_element_by_class_name('album').text)
            except:
                pass

        # artist name
            try:    
                artist_name.append(album.find_element_by_class_name('chart_detail_line1').find_element_by_class_name('artist').text)
            except:
                pass
            
        # album ids
            try:
                if '_l_' in (album.find_element_by_tag_name('img')).get_attribute('id'):
                    album_id.append((album.find_element_by_tag_name('img')).get_attribute('id'))
            except:
                pass
        
        # genres
        album_tags = clicker.find_elements_by_class_name('chart_main')
        
        for album in album_tags:

            try:
                genres = []
                album_genres = album.find_elements_by_class_name('genre')
                for genre in album_genres:
                    genres.append(genre.text)

            except:
                pass
            
            album_genre.append(genres)

        #descriptor
        album_tags = driver.find_elements_by_class_name('chart_detail_line3')

        for album in album_tags:
            try:
                descriptor.append(album.find_element_by_class_name('extra_metadata.extra_metadata_descriptors').text)
            except:
                pass

        time.sleep(random.randint(2,5))    
            
        driver.find_element_by_class_name('navlinknext').click()

    return  album_name, artist_name, descriptor, album_id, album_genre
    

In [ ]:
# Replace xxxxxxx with your details
# Prepare the scraper by logging-in
clicker = open_and_login(xxxxxxxxx, xxxxxxxx, 2)

In [ ]:
# Go to charts plage
navigate_to_charts(5, clicker)

In [ ]:
# Collect 40 pages 
albums, artists, descriptors, album_ids, genres = grab_album_metadata(40, clicker)

In [ ]:
df = pd.DataFrame({'album': albums,
                   'artist': artists,
                   'descriptors': descriptors,
                   'id': albums_ids,
                   'genre': genres})

print(df.shape)

In [ ]:
df.to_csv('RYM_Top_4000.csv',index=False)